In [ ]:
pip install datasets

In [17]:
#pip install transformers==4.29.2
pip install transformers==4.29.2

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: /apps/z_install_tree/linux-rocky8-ivybridge/gcc-12.2.0/ood-jupyterlab-3.4.8-3qopzgrtpuwifxuotyxofkeze52d6ra7/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
#pip install transformers==4.17.0

In [ ]:
pip install jiwer

In [ ]:
pip install IPython

In [ ]:
pip install IPython.display

In [ ]:
pip install torch

In [ ]:
# https://github.com/huggingface/transformers/issues/22816
pip install --upgrade accelerate

In [ ]:
# https://github.com/huggingface/transformers/issues/22816
# please do pip install git+https://github.com/huggingface/accelerate to install the dev version, or pip install accelerate -U if you are not using multi-GPUs (such as in Colab).
pip install git+https://github.com/huggingface/accelerate

In [ ]:
# https://github.com/huggingface/transformers/issues/22816
# please do pip install git+https://github.com/huggingface/accelerate to install the dev version, or pip install accelerate -U if you are not using multi-GPUs (such as in Colab).
pip install accelerate -U

In [ ]:
# Based on: 
#     https://huggingface.co/blog/fine-tune-wav2vec2-english
#     https://colab.research.google.com/github/patrickvonplaten/notebooks/blob/master/Fine_tuning_Wav2Vec2_for_English_ASR.ipynb#scrollTo=e7cqAWIayn6w
#     https://github.com/monomest/Thesis/blob/3a15f747dfd934535ffb7a02bf3fee97d9c546cb/s5/wav2vec_projects/THESIS_C/run_finetune_kids_OGI.py#L239

In [5]:
from datetime import date
from datetime import datetime

now = datetime.now()
# Print out dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("Started:", dt_string)

Started: 14/06/2023 22:14:09


In [6]:
print("\n------------ Importing libraries... ------------\n")
import pandas as pd
import random
import json
import librosa
import torch
import numpy as np

import transformers
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor
from transformers import Wav2Vec2ForCTC
from transformers import TrainingArguments
from transformers import Trainer

from datasets import load_dataset, load_metric, ClassLabel, Audio
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union


------------ Importing libraries... ------------



In [7]:
#model = Wav2Vec2ForCTC.from_pretrained('/srv/scratch/z5313567/thesis/wav2vec2/model/OGI_American/model_OGI_American_20230528')

In [8]:
print("pandas version:", pd.__version__)
print("json version:", json.__version__)
print("librosa version:", librosa.__version__)
print("Numpy version:", np.__version__)
print("Transformers version:", transformers.__version__)
print("Torch version:", torch.__version__)
print("Test cuda_device_count", torch.cuda.device_count())
print("Test cuda_is_available", torch.cuda.is_available())
print("Test get_device_name", torch.cuda.get_device_name(0))

pandas version: 2.0.1
json version: 2.0.9
librosa version: 0.10.0.post2
Numpy version: 1.24.2
Transformers version: 4.29.2
Torch version: 1.13.1+cu117
Test cuda_device_count 0
Test cuda_is_available False


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [9]:
print("\n------------------ Model arguments... ------------------\n")
# For setting model = Wav2Vec2ForCTC.from_pretrained()

set_hidden_dropout = 0.1                    # Default = 0.1
print("hidden_dropout:", set_hidden_dropout)
set_activation_dropout = 0.1                # Default = 0.1
print("activation_dropout:", set_activation_dropout)
set_attention_dropout = 0.1                 # Default = 0.1
print("attention_dropout:", set_attention_dropout)
set_feat_proj_dropout = 0.0                 # Default = 0.1
print("feat_proj_dropout:", set_feat_proj_dropout)
set_layerdrop = 0.05                        # Default = 0.1
print("layerdrop:", set_layerdrop)
set_mask_time_prob = 0.065                  # Default = 0.05
print("mask_time_prob:", set_mask_time_prob)
set_mask_time_length = 10                   # Default = 10
print("mask_time_length:", set_mask_time_length)
set_ctc_loss_reduction = "mean"             # Default = "sum"
print("ctc_loss_reduction:", set_ctc_loss_reduction)
set_ctc_zero_infinity = True               # Default = False
print("ctc_zero_infinity:", set_ctc_zero_infinity)



------------------ Model arguments... ------------------

hidden_dropout: 0.1
activation_dropout: 0.1
attention_dropout: 0.1
feat_proj_dropout: 0.0
layerdrop: 0.05
mask_time_prob: 0.065
mask_time_length: 10
ctc_loss_reduction: mean
ctc_zero_infinity: True


In [10]:
print("\n------------------ Training arguments... ------------------\n")
# For setting training_args = TrainingArguments()

set_per_device_train_batch_size = 8         # Default = 8
print("per_device_train_batch_size:", set_per_device_train_batch_size)
set_group_by_length = True                  # Default = False
print("group_by_length:", set_group_by_length)
set_gradient_accumulation_steps = 1         # Default = 1
print("gradient_accumulation_steps:", set_gradient_accumulation_steps)
set_gradient_checkpointing = True           # Default = False
print("gradient_checkpointing:", set_gradient_checkpointing)
set_weight_decay = 0.01                     # Default = 0
print("weight_decay:", set_weight_decay)
set_fp16 = True                             # Default = False
print("fp16:", set_fp16)

set_learning_rate = 0.00004                 # Default = 0.00005
print("learning_rate:", set_learning_rate)
set_lr_scheduler_type = "linear"            # Default = "linear"
print("lr_scheduler_type:", set_lr_scheduler_type )
set_adam_beta1 = 0.9                        # Default = 0.9
print("adam_beta1:", set_adam_beta1)
set_adam_beta2 = 0.98                       # Default = 0.999
print("adam_beta2:", set_adam_beta2)
set_adam_epsilon = 0.00000001               # Default = 0.00000001
print("adam_epsilon:", set_adam_epsilon)
set_warmup_ratio = 0.1                      # Default = 0.0
print("warmup_ratio:", set_warmup_ratio)

set_num_train_epochs = 22                 # Default = 3.0
print("num_train_epochs:", set_num_train_epochs)
set_max_steps = 35000                       # Default = -1, overrides epochs
print("max_steps:", set_max_steps)

set_logging_strategy = "steps"              # Default = "steps"
print("logging_strategy:", set_logging_strategy)
set_logging_steps = 1000                      # Default = 500
print("logging_steps:", set_logging_steps)
set_save_strategy = "steps"                 # Default = "steps"
print("save_strategy:", set_save_strategy)
set_save_steps = 1000                         # Default = 500
print("save_steps:", set_save_steps)
set_evaluation_strategy = "steps"           # Default = "no"
print("evaluation_strategy:", set_evaluation_strategy)
set_eval_steps = 1000                         # Optional
print("eval_steps:", set_eval_steps)
set_save_total_limit = 40                   # Optional                 
print("save_total_limit:", set_save_total_limit)

set_load_best_model_at_end = True           # Default = False
print("load_best_model_at_end:", set_load_best_model_at_end)
set_metric_for_best_model = "wer"           # Optional
print("metric_for_best_model:", set_metric_for_best_model)
set_greater_is_better = False               # Optional
print("greater_is_better:", set_greater_is_better)


------------------ Training arguments... ------------------

per_device_train_batch_size: 8
group_by_length: True
gradient_accumulation_steps: 1
gradient_checkpointing: True
weight_decay: 0.01
fp16: True
learning_rate: 4e-05
lr_scheduler_type: linear
adam_beta1: 0.9
adam_beta2: 0.98
adam_epsilon: 1e-08
warmup_ratio: 0.1
num_train_epochs: 22
max_steps: 35000
logging_strategy: steps
logging_steps: 1000
save_strategy: steps
save_steps: 1000
evaluation_strategy: steps
eval_steps: 1000
save_total_limit: 40
load_best_model_at_end: True
metric_for_best_model: wer
greater_is_better: False


In [11]:
print("\n------------------ Experiment arguments... ------------------\n")
use_checkpoint = True
print('use_checkpoint:', use_checkpoint)

training = True
print('training:', training)


------------------ Experiment arguments... ------------------

use_checkpoint: True
training: True


In [12]:
print("\n------------------ Loading files... ------------------\n")

train_df_fp = '/srv/scratch/z5313567/thesis/OGI_local/OGI_scripted_train_dataframe.csv'
print(f'Training dataset is stored at {train_df_fp}\n')
dev_df_fp = '/srv/scratch/z5313567/thesis/OGI_local/OGI_scripted_dev_dataframe.csv'
print(f'Development dataset is stored at {dev_df_fp}\n')
test_df_fp = '/srv/scratch/z5313567/thesis/OGI_local/OGI_scripted_test_dataframe.csv'
print(f'Testing dataset is stored at {test_df_fp}\n')
cache_fp = '/srv/scratch/chacmod/.cache/huggingface/datasets/Jordan-OGI-finetune'
# cache_fp = '/srv/scratch/z5313567/thesis/cache'
print(f'Cache filepath is {cache_fp}\n')
model_fp = '/srv/scratch/z5313567/thesis/wav2vec2/model/OGI_American/model_OGI_American_20230609'
print(f'Model filepath is {model_fp}\n')
vocab_fp = '/srv/scratch/z5313567/thesis/wav2vec2/vocab/OGI_American/vocab_OGI_American_20230609.json'
print(f'Vocab filepath is {vocab_fp}\n')
finetuned_result_fp = '/srv/scratch/z5313567/thesis/wav2vec2/finetuned_result/OGI_American/result_OGI_American_20230609.csv'
print(f'Fine-tuned result filepath is {finetuned_result_fp}\n')

# use_checkpoint = False -----> pretrained_mod = 'facebook/wav2vec2-base'
# use_checkpoint = True  -----> pretrained_mod = checkpoint_dir
pretrained_mod = 'facebook/wav2vec2-base'
if use_checkpoint:
    checkpoint_dir = '/srv/scratch/z5313567/thesis/wav2vec2/model/OGI_American/model_OGI_American_20230609/checkpoint-18000'
    pretrained_mod = checkpoint_dir
    print(f'Checkpoint directory is {checkpoint_dir}\n')
print(f'Pretrained model is {pretrained_mod}\n')



------------------ Loading files... ------------------

Training dataset is stored at /srv/scratch/z5313567/thesis/OGI_local/OGI_scripted_train_dataframe.csv

Development dataset is stored at /srv/scratch/z5313567/thesis/OGI_local/OGI_scripted_dev_dataframe.csv

Testing dataset is stored at /srv/scratch/z5313567/thesis/OGI_local/OGI_scripted_test_dataframe.csv

Cache filepath is /srv/scratch/chacmod/.cache/huggingface/datasets/Jordan-OGI-finetune

Model filepath is /srv/scratch/z5313567/thesis/wav2vec2/model/OGI_American/model_OGI_American_20230609

Vocab filepath is /srv/scratch/z5313567/thesis/wav2vec2/vocab/OGI_American/vocab_OGI_American_20230609.json

Fine-tuned result filepath is /srv/scratch/z5313567/thesis/wav2vec2/finetuned_result/OGI_American/result_OGI_American_20230609.csv

Checkpoint directory is /srv/scratch/z5313567/thesis/wav2vec2/model/OGI_American/model_OGI_American_20230609/checkpoint-18000

Pretrained model is /srv/scratch/z5313567/thesis/wav2vec2/model/OGI_America

In [13]:
print("\n------------------ Loading datasets... ------------------\n")

dataset = load_dataset('csv', 
                    data_files={'train': train_df_fp,
                                'dev': dev_df_fp,
                                'test': test_df_fp},
                    cache_dir=cache_fp)


------------------ Loading datasets... ------------------



Found cached dataset csv (/srv/scratch/chacmod/.cache/huggingface/datasets/Jordan-OGI-finetune/csv/default-168d650f40feead3/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 3/3 [00:00<00:00, 28.49it/s]


In [14]:
#dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [15]:
print('dataset is:', dataset)

dataset is: DatasetDict({
    train: Dataset({
        features: ['filepath', 'duration', 'speaker_id', 'transcription'],
        num_rows: 50187
    })
    dev: Dataset({
        features: ['filepath', 'duration', 'speaker_id', 'transcription'],
        num_rows: 10965
    })
    test: Dataset({
        features: ['filepath', 'duration', 'speaker_id', 'transcription'],
        num_rows: 10847
    })
})


In [16]:
print("\n------------------ Showing some random elements... ------------------\n")

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), f"Can't pick more elements than there are in the dataset {len(dataset)}." 
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    print(df)


------------------ Showing some random elements... ------------------



In [17]:
show_random_elements(dataset["train"].remove_columns(["filepath", "duration", "speaker_id"]), num_examples=10)

                           transcription
0                              advantage
1  why is the earth inside the milky way
2           i thought i heard new voices
3                                engrave
4                                   four
5                                   sour
6                                 spoons
7                      take off your hat
8                             stepfather
9                                ragtime


In [18]:
print("\n------------------ Extracting individual characters... ------------------\n")
def extract_all_chars(batch):
    all_text = " ".join(batch["transcription"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}


------------------ Extracting individual characters... ------------------



In [19]:
vocabs = dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset.column_names["train"])

In [20]:
print('vocabs is:', vocabs)

vocabs is: DatasetDict({
    train: Dataset({
        features: ['vocab', 'all_text'],
        num_rows: 1
    })
    dev: Dataset({
        features: ['vocab', 'all_text'],
        num_rows: 1
    })
    test: Dataset({
        features: ['vocab', 'all_text'],
        num_rows: 1
    })
})


In [21]:
#dataset.column_names["train"]

In [22]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

In [23]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
print("vocab_dict is:", vocab_dict)

vocab_dict is: {'h': 0, 's': 1, 'k': 2, 'c': 3, 't': 4, 'e': 5, 'l': 6, 'a': 7, 'f': 8, 'g': 9, 'r': 10, 'b': 11, 'd': 12, 'i': 13, ' ': 14, 'm': 15, 'z': 16, 'o': 17, 'w': 18, 'q': 19, 'y': 20, 'j': 21, "'": 22, 'x': 23, 'v': 24, 'u': 25, 'p': 26, 'n': 27}


In [24]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(f'The lendth of vocab_list is {len(vocab_dict)}')

The lendth of vocab_list is 30


In [25]:
with open(vocab_fp, 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)
print('Successfully created vocab.json file at:', vocab_fp)

Successfully created vocab.json file at: /srv/scratch/z5313567/thesis/wav2vec2/vocab/OGI_American/vocab_OGI_American_20230609.json


In [26]:
print("\n------------------ Creating tokenizer... ------------------\n")
tokenizer = Wav2Vec2CTCTokenizer(vocab_fp, unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")


------------------ Creating tokenizer... ------------------



In [27]:
print("\n------------------ Creating feature extractor... ------------------\n")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)


------------------ Creating feature extractor... ------------------



In [28]:
print("\n------------------ Creating processor... ------------------\n")
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
processor.save_pretrained(model_fp)


------------------ Creating processor... ------------------



In [29]:
def process_dataset(batch):
    speech_array, sampling_rate = librosa.load(batch['filepath'], sr=feature_extractor.sampling_rate)
    batch['speech'] = speech_array
    batch['sampling_rate'] = sampling_rate
    batch["target_text"] = batch["transcription"]
    return batch


In [30]:
print("\n------------------ Obtaining speech arrays, sampling rates, and target texts... ------------------\n")
dataset = dataset.map(process_dataset, remove_columns=dataset.column_names["train"], num_proc=4)


------------------ Obtaining speech arrays, sampling rates, and target texts... ------------------



                                                                             
KeyboardInterrupt



In [42]:
print('dataset is:', dataset)

dataset is: DatasetDict({
    train: Dataset({
        features: ['speech', 'sampling_rate', 'target_text'],
        num_rows: 50187
    })
    dev: Dataset({
        features: ['speech', 'sampling_rate', 'target_text'],
        num_rows: 10965
    })
    test: Dataset({
        features: ['speech', 'sampling_rate', 'target_text'],
        num_rows: 10847
    })
})


In [28]:
# the sampling rate of the data that was used to pretrain the model should match the sampling rate of the dataset used to fine-tune the model.
def check_sampling_rate(dataset): # say, dataset = dataset['train']
    target_sr = feature_extractor.sampling_rate
    if len(set(dataset['sampling_rate'])) == 1:
        actual_sr = list(set(dataset['sampling_rate']))[0] # sampling rate = 22050 Hz for OGI corpus
        if actual_sr != target_sr:
            print('MISMATCH!: the sampling rate used for fine-tuning Wav2vec2.0 does not match the sampling rate used for pretraining')
            
            now = datetime.now()
            # Print out dd/mm/YY H:M:S
            # ------------------------------------------
            dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
            print('Resampling starts...', dt_string)
            for i in range(len(dataset)):
                dataset[i]['speech'] = librosa.resample(np.asarray(dataset[i]['speech']), orig_sr=actual_sr, target_sr=target_sr)
            now = datetime.now()
            # dd/mm/YY H:M:S
            dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
            print('Resampling completes', dt_string)
        else:
            print('MATCH!: The sampling rate used for fine-tuning Wav2vec2.0 matches the sampling rate used for pretraining')    

In [29]:
print('\n------------------Ckech sampling rates of training datasets... ------------------\n')
check_sampling_rate(dataset['train'])


------------------Ckech sampling rates of training datasets... ------------------

MATCH!: The sampling rate used for fine-tuning Wav2vec2.0 matches the sampling rate used for pretraining


In [30]:
print('\n------------------Ckeck sampling rates of development datasets... ------------------\n')
check_sampling_rate(dataset['dev'])


------------------Ckeck sampling rates of development datasets... ------------------

MATCH!: The sampling rate used for fine-tuning Wav2vec2.0 matches the sampling rate used for pretraining


In [31]:
print('\n------------------Ckech sampling rates of testing datasets... ------------------\n')
check_sampling_rate(dataset['test'])


------------------Ckech sampling rates of testing datasets... ------------------

MATCH!: The sampling rate used for fine-tuning Wav2vec2.0 matches the sampling rate used for pretraining


In [32]:
#print(list(set(dataset['train']['sampling_rate']))[0])
#print(len(set(dataset['train']['sampling_rate'])))

In [33]:
print("\n------------------ Verifying some ramdom samples... ------------------\n")
# verify if the data is a 1-dimensional array, the sampling rate corresponds to 16kHz, and the target text is clean.
for i in range(6):
    rand_int = random.randint(0, len(dataset["train"])-1)
    #rand_int = i
    print('Target text:', dataset['train'][rand_int]['target_text'])
    print('Input array shape:', np.asarray(dataset['train'][rand_int]['speech']).shape)
    print('Sampling rate:', dataset['train'][rand_int]["sampling_rate"])
    print('\n')


------------------ Verifying some ramdom samples... ------------------

Target text: four
Input array shape: (33223,)
Sampling rate: 16000


Target text: throughout
Input array shape: (25404,)
Sampling rate: 16000


Target text: even your ears sweat in a sauna
Input array shape: (73804,)
Sampling rate: 16000


Target text: nine
Input array shape: (50506,)
Sampling rate: 16000


Target text: women
Input array shape: (41748,)
Sampling rate: 16000


Target text: biology
Input array shape: (50502,)
Sampling rate: 16000




In [34]:
def prepare_dataset(batch):
    #batch["input_values"] = processor(batch['speech'], sampling_rate=batch["sampling_rate"]).input_values[0]
    batch["input_values"] = processor(batch['speech'], sampling_rate=batch["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [35]:
print("\n------------------ Obtaining input values, input length, and labels... ------------------\n")
#prep_dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names["train"], num_proc=4)
# dataset = dataset.map(prepare_dataset, num_proc=4)
data_prepared = dataset.map(prepare_dataset, remove_columns=dataset.column_names["train"], batch_size=8, num_proc=4, batched=True)


------------------ Obtaining input values, input length, and labels... ------------------



Map (num_proc=4):   0%|          | 0/50187 [00:00<?, ? examples/s]/home/z5313567/.local/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/z5313567/.local/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/z5313567/.local/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in 

In [ ]:
#print(prep_dataset)
print('dataset is:', dataset)

In [ ]:
#---------------Training + Evaluation--------------------
# ** Define a padding data collator used to pad the training samples to longest sample in their batch. In contrast to most NLP models,
#    Wav2Vec2 has a much larger input length than output length. E.g., a sample of input length 50000 has an output length of no more than 100. 
#    Given the large input sizes, it is much more efficient to pad the training batches dynamically meaning that all training samples should only 
#    be padded to the longest sample in their batch and not the overall longest sample. Therefore, fine-tuning Wav2Vec2 requires a special padding data collator, 
#    which we will define below
# ** Evaluation metric. During training, the model should be evaluated on the word error rate. We should define a compute_metrics function accordingly
# ** Load a pretrained checkpoint. We need to load a pretrained checkpoint and configure it correctly for training.
# ** Define the training configuration.

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None
    
    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace label ids of padding tokens with -100 so that those tokens are not taken into account when computing the loss
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
print('\n------------------ Setting up the padding data collator... ------------------\n')
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
print('\n------------------ Setting up WER metric... ------------------\n')
wer_metric = load_metric("wer")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [94]:
print('\n------------------ Loading a pretrained checkpount... ------------------\n')
model = Wav2Vec2ForCTC.from_pretrained(
    pretrained_mod, 
    vocab_size=len(processor.tokenizer),
    pad_token_id=processor.tokenizer.pad_token_id,
    hidden_dropout=set_hidden_dropout,
    activation_dropout=set_activation_dropout,
    attention_dropout=set_attention_dropout,
    feat_proj_dropout=set_feat_proj_dropout,
    layerdrop=set_layerdrop,
    mask_time_prob=set_mask_time_prob,
    mask_time_length=set_mask_time_length,
    ctc_loss_reduction=set_ctc_loss_reduction,
    ctc_zero_infinity=set_ctc_zero_infinity    
)


------------------ Loading a pretrained checkpount... ------------------



/home/z5313567/.local/lib/python3.10/site-packages/transformers/configuration_utils.py:356: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForCTC: ['project_hid.bias', 'quantizer.codevectors', 'project_q.bias', 'project_q.weight', 'quantizer.weight_proj.weight', 'project_hid.weight', 'quantizer.weight_proj.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model t

In [95]:
# CNN layers of Wav2vec2.0 model is sufficiently trained, hence they do not need to be finetuned anymore
model.freeze_feature_encoder()

In [96]:
print('\n------------------ Setting TrainingArguments... ------------------\n')
training_args = TrainingArguments(
    output_dir=model_fp,
    per_device_train_batch_size=set_per_device_train_batch_size,
    group_by_length=set_group_by_length,
    gradient_accumulation_steps=set_gradient_accumulation_steps,
    gradient_checkpointing=set_gradient_checkpointing,
    weight_decay=set_weight_decay,
    fp16=set_fp16,
    learning_rate=set_learning_rate,
    lr_scheduler_type=set_lr_scheduler_type,
    adam_beta1=set_adam_beta1,
    adam_beta2=set_adam_beta2,
    adam_epsilon=set_adam_epsilon,
    warmup_ratio=set_warmup_ratio,
    num_train_epochs=set_num_train_epochs,
    max_steps=set_max_steps,
    logging_strategy=set_logging_strategy,
    logging_steps=set_logging_steps,
    save_strategy=set_save_strategy,
    save_steps=set_save_steps,
    evaluation_strategy=set_evaluation_strategy,
    eval_steps=set_eval_steps,
    save_total_limit=set_save_total_limit,
    load_best_model_at_end=set_load_best_model_at_end,
    metric_for_best_model=set_metric_for_best_model,
    greater_is_better=set_greater_is_better
)

In [97]:
print('\n------------------ Setting Trainer... ------------------\n')

'''
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset["train"],
    eval_dataset=dataset["dev"],
    tokenizer=processor.feature_extractor,
)
'''
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=data_prepared["train"],
    eval_dataset=data_prepared["dev"],
    tokenizer=processor.feature_extractor,
)


In [ ]:
if training:
    print('\n------------------ Starting training... ------------------\n')
    torch.cuda.empty_cache()
    if use_checkpoint:
        trainer.train(resume_from_checkpoint=checkpoint_dir)
    else:
        trainer.train()
    model.save_pretrained(model_fp)
    
       
    now = datetime.now()
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    print("Training Finished:", dt_string)
    print('\n------------------ Training finished... ------------------\n')

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
/home/z5313567/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 50187
  Num Epochs = 30
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 188220


Step,Training Loss,Validation Loss


In [ ]:
print('\n------------------ Evaluation starts... ------------------\n')
processor = Wav2Vec2Processor.from_pretrained(model_fp)
model = Wav2Vec2ForCTC.from_pretrained(model_fp)

In [ ]:
# Now, we will make use of the map(...) function to predict the transcription of every test sample and to save the prediction 
# in the dataset itself. We will call the resulting dictionary "results".

# Note: we evaluate the test data set with batch_size=1 on purpose due to this issue. Since padded inputs don't yield the 
# exact same output as non-padded inputs, a better WER can be achieved by not padding the input at all.

In [ ]:
print('\n------------------ Generating fine-tuned results... ------------------\n')


def map_to_result(batch):
    model.to("cuda")
    input_values = processor(
      batch["speech"], 
      sampling_rate=batch["sampling_rate"], 
      return_tensors="pt"
    ).input_values.to("cuda")

    with torch.no_grad():
        logits = model(input_values).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  
    return batch


In [ ]:
'''
def map_to_result(batch):
    with torch.no_grad():
        input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
        logits = model(input_values).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  
    return batch
    '''

In [ ]:
results = dataset["test"].map(map_to_result)
print('results are:', results)

results_df = results.to_pandas()
results_df = results_df.drop(columns=["speech", "sampling_rate", "input_values", "input_length", "labels"])
results_df.to_csv(finetuned_result_fp)
print('Fine-tuned results are saved to:', finetuned_result_fp)

In [ ]:
print('\n------------------ Generating WER result on test dataset... ------------------\n')
print("Fine-tuned Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["target_text"])))

In [ ]:
print('\n------------------ Showing some random prediction errors... ------------------\n')
show_random_elements(results.remove_columns(["speech", "sampling_rate", "input_values", "input_length", "labels"]), num_examples=10)

In [ ]:
print('\n------------------ Generating the exact output of the model... ------------------\n')

model.to("cuda")
input_values = processor(
      dataset["test"][0]["speech"], 
      sampling_rate=dataset["test"][0]["sampling_rate"], 
      return_tensors="pt"
  ).input_values.to("cuda")

with torch.no_grad():
    logits = model(input_values).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
print(" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist())))

In [ ]:
'''
model.to("cuda")

with torch.no_grad():
    logits = model(torch.tensor(dataset["test"][:1]["input_values"], device="cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
print(" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist())))
'''

In [ ]:
now = datetime.now()
# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("Evaluation Finished:", dt_string)
print('\n------------------ Evaluation finished... ------------------\n')